In [ ]:
"""
!pip install -q sciunit
!pip install -q neo
!pip install -q elephant
!pip install -q gitpython
!pip install -q efel
"""

In [ ]:
#!pip install scipy seaborn
!pip install -U git+https://github.com/appukuttan-shailesh/hbp-validation-client.git
#!pip install --quiet fpdf PyPDF2 tabulate
#!pip install ./bbp-services-0.0.15/ ./task-types-0.0.10/ 
!pip install hbp_service_client

In [ ]:
# Register a new test on the test library
name="NeuroM morpho-stats Test"
alias="morph_stats_Test"
version="1.0"
author="Pedro Garcia-Rodriguez"
species="Undefined"
age="Any"
brain_region="Striatum"
cell_type="Other"
data_modality="electron microscopy"
test_type="network structure"
score_type="Other"
protocol="Tests a set of cell's morphological features in a digitally reconstructed neuron"
data_location="collab://8123/test_observations/morphounit/FSI_morpho_features_NeuroM.json"
data_type="Morphology"
publication="Cutuli (2011), DOI:10.1016/j.nbd.2010.12.007, Ransome 92005), DOI:10.1016/j.neuroscience.2004.12.041, Tepper (2010), DOI:10.3389/fnana.2010.00150, Gittis (2011), DOI:10.1016/j.neuron.2011.06.035"
repository="https://github.com/pedroernesto/morphounit.git"
path="morphounit.tests.NeuroM_MorphStats_Test"

from hbp_validation_framework import TestLibrary
HBP_USERNAME = "pedroernesto"
test_library = TestLibrary(username=HBP_USERNAME, environment="dev")

test = test_library.add_test(name=name, alias=alias, version=version, author=author,
                        species=species, age=age, brain_region=brain_region, cell_type=cell_type,
                        data_modality=data_modality, test_type=test_type, score_type=score_type,
                        data_location=data_location, data_type=data_type, publication=publication,
                        repository=repository, path=path)


In [ ]:
# Create a new model entry in the ModelCatalog; metadata to be used for registering model
from hbp_validation_framework import ModelCatalog, TestLibrary
HBP_USERNAME = "pedroernesto"
env="production"
testLibrary = TestLibrary(username=HBP_USERNAME, environment=env)
model_catalog = ModelCatalog.from_existing(testLibrary)

model = model_catalog.register_model(app_id="39968", name="Reconstructed cell's morphology",
                alias="NeuroM-MorphStats-Model", author="Alexander Koslov", organization="HBP-SP6",
                private=True, cell_type="Fast spiking interneuron", model_type="Single Cell",
                brain_region="Striatum", species="Other",
                instances=[{"source":"https://github.com/pedroernesto/VF_test_models",
                            "version":"1.0", "parameters":""}],
                description="A class to interact with morphology files via the morphometrics-NeuroM's API (morph_stats)")

In [ ]:
import models
models_path = 'str-fs_pv-20170919'
# models_path = './str-fs_excert'
config_path = './FSI_morph_stats_config.yaml'
stats_file = 'NeuroM_MorphStats_prediction.json'

# model2Test = models.CA1_laminar_distribution_synapses()

In [ ]:
!pip uninstall --yes morphounit
!pip install --upgrade --quiet git+https://github.com/pedroernesto/morphounit.git

import os
obs_path = os.path.expanduser('~/Documents/Project/Code/Models_Validation/Validation/ValidationTests/morphounit_expData')
obs_file = os.path.join(obs_path, 'FSI_morpho_stats_NeuroM.json')

import json
fp = open(obs_file, 'r+')
observation = json.load(fp)
fp.close()

import morphounit.tests

test = morphounit.tests.NeuroM_MorphStats_Test(observation=observation)

In [ ]:
# First login in into the production version of the VF
from hbp_validation_framework import utils, ModelCatalog
HBP_USERNAME = "pedroernesto"
x=ModelCatalog(username=HBP_USERNAME)

result_uuids = []
test_instance_id='5d7334d7-a406-470a-a8fe-2d10a88ec4e7'
morph_files = os.listdir(models_path)
for file_name in morph_files:
    morph_path = os.path.join(models_path, file_name)
    model_name = (file_name.split("/")[-1]).split(".")[0] # Cell-ID
                  
    model2Test = models.NeuroM_MorphStats(model_name=model_name, morph_path=morph_path, 
                                          config_path=config_path, morph_stats_file=stats_file)

    # print morph_path, "---", model_name, '\n\n'
    # score = test.judge(model2Test, deep_error=True)

    result_id = utils.run_test(environment="dev", hbp_username="pedroernesto", model=model2Test, 
                   test_instance_id=test_instance_id, test_version="2.0")
    result_uuids.append(result_id)

In [ ]:
report_path = None
if len(result_uuids) > 0:
    print("\nDo you wish to generate a report of the tests executed?")
    print("Enter: y/n")
    choice = raw_input().lower()
    valid_choices = {"yes": True, "y": True, "no": False, "n": False}
    if valid_choices[choice]:
        valid_uuids, report_path = utils.generate_report(hbp_username=HBP_USERNAME, environment="dev", 
                                                         result_list=result_uuids, only_combined=True)

In [ ]:
print report_path
if report_path:
    rel_report_path = os.path.relpath(report_path)
    from IPython.display import IFrame    
    display(IFrame(rel_report_path, width=900, height=650))